In [26]:
import os
from pymongo import MongoClient
conn_url = "mongodb://localhost:27017/"

try:
    client = MongoClient(conn_url)
    db = client["your_database_name"]  # Choose your database name
    collection = db["chat_history"]  # Choose your collection name
    print("Successfully connected to MongoDB.")
except Exception as e:
    print(f"Error connecting to MongoDB: {e}")
    # Handle the error appropriately, e.g., exit or use a fallback mechanism\
client.close()

Successfully connected to MongoDB.


In [6]:
from datetime import datetime
from fastapi import FastAPI, HTTPException, Depends
# ... other imports

def chat_endpoint(user_id: str, message: str): # Add session_id if needed
    try:
        # 1. Store the user's message
        collection.insert_one({
            "user_id": user_id,
            "timestamp": datetime.now(),
            "role": "user",
            "content": message
        })

        # 2. Get the chat history (last 10 messages)
        chat_history = list(collection.find(
            {"user_id": user_id},
            sort=[("timestamp", -1)],  # Sort by timestamp, newest first
            limit=10
        ))
        chat_history.reverse() # reverse the order so that the oldest message comes first.

        # 3. Use chat_history with your LangChain LLM
        # ... your LangChain logic here, using chat_history ...
        # Example (adapt to your specific LLM setup):
        # llm_response = your_llm(message, history=chat_history)

        print(chat_history)

        # 4. Store the assistant's response
        collection.insert_one({
            "user_id": user_id,
            "timestamp": datetime.now(),
            "role": "assistant",
            "content": "llm_response" # replace with actual response.
        })

        return {"response": "llm_response","history":chat_history}

    except Exception as e:
        print(f"Error in chat endpoint: {e}")
        raise HTTPException(status_code=500, detail="An error occurred during chat.")

In [8]:
out = chat_endpoint("1","sampl message")

[{'_id': ObjectId('67b59fa76f3aa722783be9aa'), 'user_id': '1', 'timestamp': datetime.datetime(2025, 2, 19, 14, 38, 55, 420000), 'role': 'user', 'content': 'sampl message'}, {'_id': ObjectId('67b59fa76f3aa722783be9ab'), 'user_id': '1', 'timestamp': datetime.datetime(2025, 2, 19, 14, 38, 55, 438000), 'role': 'assistant', 'content': 'llm_response'}, {'_id': ObjectId('67b59fe26f3aa722783be9ac'), 'user_id': '1', 'timestamp': datetime.datetime(2025, 2, 19, 14, 39, 54, 941000), 'role': 'user', 'content': 'sampl message'}, {'_id': ObjectId('67b59fe26f3aa722783be9ad'), 'user_id': '1', 'timestamp': datetime.datetime(2025, 2, 19, 14, 39, 54, 943000), 'role': 'assistant', 'content': 'llm_response'}, {'_id': ObjectId('67b5a0436f3aa722783be9ae'), 'user_id': '1', 'timestamp': datetime.datetime(2025, 2, 19, 14, 41, 31, 895000), 'role': 'user', 'content': 'sampl message'}, {'_id': ObjectId('67b5a0436f3aa722783be9af'), 'user_id': '1', 'timestamp': datetime.datetime(2025, 2, 19, 14, 41, 31, 898000), 'rol

In [19]:
out = chat_history = list(collection.find(
            {"user_id": "1"},
            sort=[("timestamp", -1)],  # Sort by timestamp, newest first
            limit=10
        ))
chat_history.reverse()
out

[{'_id': ObjectId('67b59fa76f3aa722783be9aa'),
  'user_id': '1',
  'timestamp': datetime.datetime(2025, 2, 19, 14, 38, 55, 420000),
  'role': 'user',
  'content': 'sampl message'},
 {'_id': ObjectId('67b59fa76f3aa722783be9ab'),
  'user_id': '1',
  'timestamp': datetime.datetime(2025, 2, 19, 14, 38, 55, 438000),
  'role': 'assistant',
  'content': 'llm_response'},
 {'_id': ObjectId('67b59fe26f3aa722783be9ac'),
  'user_id': '1',
  'timestamp': datetime.datetime(2025, 2, 19, 14, 39, 54, 941000),
  'role': 'user',
  'content': 'sampl message'},
 {'_id': ObjectId('67b59fe26f3aa722783be9ad'),
  'user_id': '1',
  'timestamp': datetime.datetime(2025, 2, 19, 14, 39, 54, 943000),
  'role': 'assistant',
  'content': 'llm_response'},
 {'_id': ObjectId('67b5a0436f3aa722783be9ae'),
  'user_id': '1',
  'timestamp': datetime.datetime(2025, 2, 19, 14, 41, 31, 895000),
  'role': 'user',
  'content': 'sampl message'},
 {'_id': ObjectId('67b5a0436f3aa722783be9af'),
  'user_id': '1',
  'timestamp': datetim

In [21]:
print(" ".join([chat["role"]+":"+chat["content"]+"\n" for chat in out]))

user:sampl message
 assistant:llm_response
 user:sampl message
 assistant:llm_response
 user:sampl message
 assistant:llm_response
 user:sampl message
 assistant:llm_response



In [23]:
def chat(query:str):

    
    # If the user is an authenticated user
    user_id = 1
    
    # if chat_history_model is not None:
    #     print("existing user")
    #     history_collection = list(collection.find(
    #         {"user_id": user_id},
    #         sort=[("timestamp", -1)],  # Sort by timestamp, newest first
    #         limit=10
    #     ))
    #     history_collection.reverse()
    print("New user")
    history_collection = None

    if history_collection:
        chat_message_history = " ".join([chat["role"]+":"+chat["content"]+"\n" for chat in history_collection])
    else:
        chat_message_history = ""

    print("CHAT MESSAGE HISORY:",chat_message_history)

    # Get the LLM response for the query with history
    llm_response = "Hello there"
    print(llm_response)
    #store the chat message information to the sqlite db


    #Store the updated chat history to Mongodb
    collection.insert_one({
            "user_id": user_id,
            "timestamp": datetime.now(),
            "role": "user",
            "content": query
        })
    collection.insert_one({
        "user_id": user_id,
        "timestamp": datetime.now(),
        "role": "assistant",
        "content": llm_response  # replace with actual response.
    })
    # except:
    #     raise HTTPException(status_code=500, detail="An error occurred during chat.")

    return llm_response


In [24]:
chat("HI!")

New user
CHAT MESSAGE HISORY: 
Hello there


'Hello there'